Đường dẫn đến dataset trên kaggle: https://www.kaggle.com/START-UMD/gtd

#  Thư viện

In [8]:

import pandas as pd
import numpy as np

# Thư viện dùng để trực quan hóa dữ liệu
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

# Thư viện dùng để hiển thị bản đồ thế giới 
#pip install folium
#pip install https://github.com/matplotlib/basemap/archive/master.zip
# import folium
# import folium.plugins
# from mpl_toolkits.basemap import Basemap

# import io
# import base64
# from IPython.display import HTML, display
# from matplotlib import animation,rc

%matplotlib inline
sns.set(rc={"figure.figsize": (16, 8)})

# Đọc dữ liệu

In [9]:
# Đọc dữ liệu, sử dụng data frame của pandas
original_df = pd.read_csv("globalterrorismdb_0718dist.csv", encoding='ISO-8859-1')
#copy dữ liệu ra ngoài
df = original_df.copy()
# Xem thử 1 số dòng đầu của dữ liệu
df.head()

c:\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (4,6,31,33,61,62,63,76,79,90,92,94,96,114,115,121) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaN,58,Dominican Republic,2,...,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaN,130,Mexico,1,...,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaN,160,Philippines,5,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaN,78,Greece,8,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
4,197001000003,1970,1,0,NaN,0,NaN,101,Japan,4,...,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN


In [10]:
# xem thử 1 số dòng cuối của dữ liệu
df.tail()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,...,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
181686,201712310022,2017,12,31,NaN,0,NaN,182,Somalia,11,...,NaN,"""Somalia: Al-Shabaab Militants Attack Army Che...","""Highlights: Somalia Daily Media Highlights 2 ...","""Highlights: Somalia Daily Media Highlights 1 ...",START Primary Collection,0,0,0,0,NaN
181687,201712310029,2017,12,31,NaN,0,NaN,200,Syria,10,...,NaN,"""Putin's 'victory' in Syria has turned into a ...","""Two Russian soldiers killed at Hmeymim base i...","""Two Russian servicemen killed in Syria mortar...",START Primary Collection,-9,-9,1,1,NaN
181688,201712310030,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Maguindanao clashes trap tribe members,"" Phil...",NaN,NaN,START Primary Collection,0,0,0,0,NaN
181689,201712310031,2017,12,31,NaN,0,NaN,92,India,6,...,NaN,"""Trader escapes grenade attack in Imphal,"" Bus...",NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN
181690,201712310032,2017,12,31,NaN,0,NaN,160,Philippines,5,...,NaN,"""Security tightened in Cotabato following IED ...","""Security tightened in Cotabato City,"" Manila ...",NaN,START Primary Collection,-9,-9,0,-9,NaN


In [11]:
# Lấy số lượng cột của dữ liệu
len(df.columns)

135

In [12]:
# Lấy số lượng dòng của dữ liệu
num_rows = len(df)
num_rows

181691

In [13]:
#Xác định số lượng dữ liệu bị thiếu ở mỗi cột
df.isnull().sum() 

eventid            0
iyear              0
imonth             0
iday               0
approxdate    172452
               ...  
INT_LOG            0
INT_IDEO           0
INT_MISC           0
INT_ANY            0
related       156653
Length: 135, dtype: int64

Quan sát qua số lượng cột dữ liệu ta thấy quá lớn và có quá nhiều thuộc tính chứa giá trị null hoặc nan

# Tiền xử lý dữ liệu

Loại bỏ các cột có giá trị **nan**

In [14]:
# bỏ đi các cột có số lượng giá trị nan lên tới 80% dữ liệu
cutoff = 0.8
df = df.dropna(thresh=int(cutoff*num_rows), axis=1)
# hiển thị số cột còn lại
len(df.columns)

47

In [15]:
# thống kê số lượng giá trị nan cho các cột còn lại
df.isnull().sum()

eventid                 0
iyear                   0
imonth                  0
iday                    0
extended                0
country                 0
country_txt             0
region                  0
region_txt              0
provstate             421
city                  434
latitude             4556
longitude            4557
specificity             6
vicinity                0
crit1                   0
crit2                   0
crit3                   0
doubtterr               1
multiple                1
success                 0
suicide                 0
attacktype1             0
attacktype1_txt         0
targtype1               0
targtype1_txt           0
targsubtype1        10373
targsubtype1_txt    10373
target1               636
natlty1              1559
natlty1_txt          1559
gname                   0
guncertain1           380
individual              0
weaptype1               0
weaptype1_txt           0
weapsubtype1        20768
weapsubtype1_txt    20768
nkill       

Quan sát qua 1 lần nữa các cột sau khi đã lược bỏ, đánh giá từng cột như sau:
- Các cột không có thông tin để phân tích do không được mô tả cần loại bỏ gồm có:
  + INT_*
  + dbsource
  + crit* 
  + specificity 
  + vicinity 
  + doubtterr 
  + multiple
  + natlty1
  + guncertain1
  + individual
  + property
  + ishostkid
  + dbsource
  + success
  + eventid 
- Các cột cần bỏ do thông tin không cần thiết hoặc bị trùng lặp:
  + region (bỏ vì **region** chỉ chứa code định danh, còn **region_txt** mới là tên cần lấy)
  + country (đã có **country_txt** mô tả)
  + provstate (quận huyện không cần thiết, đã có tên thành phố trong thuộc tính **city**)
  + attacktype1 (đã có **attacktype1_txt** mô tả)
  + targtype1, targtype1_txt (đã có **target1** mô tả)
  + targsubtype1, targsubtype1_txt (đã có **target1** mô tả)
  + natlty1_txt (quốc gia được nhắm tới khớp với quốc gia xảy ra bạo loạn)
  + weaptype1 (đã có  **weaptype1_txt** mô tả)
  + weapsubtype1 (dư)
  + weapsubtype1_txt (dư)


 **Sau đó thay thế các tên thành chữ cho dễ đọc**

In [16]:
# Loại bỏ các cột đã định
no_information_columns = ['eventid', 'INT_LOG', 'INT_IDEO', 'INT_MISC', 'INT_ANY', 'dbsource', 'crit1', 'crit2', 'crit3', 'specificity', 'vicinity', 'doubtterr',
                          'multiple', 'natlty1', 'guncertain1', 'individual', 'property', 'ishostkid', 'dbsource', 'success']
duplicate_information_columns = ['region', 'country', 'provstate', 
                                 'attacktype1', 'targtype1', 'targtype1_txt','targsubtype1', 'targsubtype1_txt', 
                                 'natlty1_txt', 'weaptype1', 'weapsubtype1', 'weapsubtype1_txt']    
df.drop(no_information_columns, axis=1, inplace=True)
df.drop(duplicate_information_columns, axis=1, inplace=True)
df.head()

,iyear,imonth,iday,extended,country_txt,region_txt,city,latitude,longitude,suicide,attacktype1_txt,target1,gname,weaptype1_txt,nkill,nwound
0,1970,7,2,0,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,0,Assassination,Julio Guzman,MANO-D,Unknown,1.0,0.0
1,1970,0,0,0,Mexico,North America,Mexico city,19.371887,-99.086624,0,Hostage Taking (Kidnapping),"Nadine Chaval, daughter",23rd of September Communist League,Unknown,0.0,0.0
2,1970,1,0,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,0,Assassination,Employee,Unknown,Unknown,1.0,0.0
3,1970,1,0,0,Greece,Western Europe,Athens,37.997490,23.762728,0,Bombing/Explosion,U.S. Embassy,Unknown,Explosives,NaN,NaN
4,1970,1,0,0,Japan,East Asia,Fukouka,33.580412,130.396361,0,Facility/Infrastructure Attack,U.S. Consulate,Unknown,Incendiary,NaN,NaN


In [17]:
len(df.columns)

16

In [18]:
df.rename(columns={'iyear':'year','imonth':'month', 'iday':'day', 'extended':'extend',
                      'country_txt':'country','region_txt':'region','city':'city',
                      'attacktype1_txt':'attack_type','target1':'target',
                      'nkill':'killed','nwound':'wounded',
                      'gname':'group',
                      'weaptype1_txt':'weapon_type',
                      },inplace=True)

df.head()

,year,month,day,extend,country,region,city,latitude,longitude,suicide,attack_type,target,group,weapon_type,killed,wounded
0,1970,7,2,0,Dominican Republic,Central America & Caribbean,Santo Domingo,18.456792,-69.951164,0,Assassination,Julio Guzman,MANO-D,Unknown,1.0,0.0
1,1970,0,0,0,Mexico,North America,Mexico city,19.371887,-99.086624,0,Hostage Taking (Kidnapping),"Nadine Chaval, daughter",23rd of September Communist League,Unknown,0.0,0.0
2,1970,1,0,0,Philippines,Southeast Asia,Unknown,15.478598,120.599741,0,Assassination,Employee,Unknown,Unknown,1.0,0.0
3,1970,1,0,0,Greece,Western Europe,Athens,37.997490,23.762728,0,Bombing/Explosion,U.S. Embassy,Unknown,Explosives,NaN,NaN
4,1970,1,0,0,Japan,East Asia,Fukouka,33.580412,130.396361,0,Facility/Infrastructure Attack,U.S. Consulate,Unknown,Incendiary,NaN,NaN


Thống kê lại các thuộc tính:
1. **year**: năm xảy vụ khủng bố
2. **month**: tháng trong năm xảy ra khủng bố
3. **day**: ngày trong tháng xảy ra khủng bố
4. **extend**: bằng 1 nếu thời gian khủng bố kéo dài hơn 24 tiếng, ngược lại bằng 0
5. **country**: tên quốc gia xảy ra khủng bố
6. **region**: khu vực xảy ra khủng bố
7. **city**: thành phố hoặc làng hoặc thị trấn xảy ra khủng bố
8. **latitude**: vĩ độ
9. **longitude**: kinh độ
10. **suicide**: bằng 1 nếu là cuộc tấn công tự sát, ngược lại bằng 0
11. **attack_type**: hình thức tấn công
12. **target**: đối tượng bị tấn công
13. **group**: nhóm thực hiện tấn công
14. **weapon_type**: vũ khí thực hiện tấn công
15. **killed**: số lượng người chết
16. **wounded**: số lượng người bị thương


# Các câu hỏi thống kê

Câu 1: Bảng số liệu này được thu thập trong khoảng thời gian nào? Có bao nhiêu cuộc khủng bố xảy ra?

In [19]:
print(f'Thời gian từ {df.year.min()} đến {df.year.max()}')
print(f'Số cuộc khủng bố là: {df.index.nunique()}')

Thời gian từ 1970 đến 2017
Số cuộc khủng bố là: 181691


Câu 2: Có bao nhiêu quốc gia xảy ra khủng bố? Bao gồm bao nhiêu khu vực?

In [20]:
print(f'Số lượng quốc gia xảy ra khủng bố là: {df.country.nunique()}')
print(f'Trải rộng trên {df.region.nunique()} khu vực')

Số lượng quốc gia xảy ra khủng bố là: 205
Trải rộng trên 12 khu vực


Câu 3: Số lượng người chết và bị thương?

In [21]:
print(f'Số lượng người bị thương là {int(df.killed.sum())} đến {int(df.wounded.sum())}')

Số lượng người bị thương là 411868 đến 523869


Câu 4: Đâu là nơi xảy ra tấn công nhiều nhất? Bao nhiêu cuộc tấn công?

In [22]:
print('Danh sách 10 quốc gia bị tấn công nhiều nhất:')
for i in range(0,10):
    print(f"{i+1}. {df.country.value_counts().index[i]} với {df.country.value_counts()[i]} cuộc tấn công")

Danh sách 10 quốc gia bị tấn công nhiều nhất:
1. Iraq với 24636 cuộc tấn công
2. Pakistan với 14368 cuộc tấn công
3. Afghanistan với 12731 cuộc tấn công
4. India với 11960 cuộc tấn công
5. Colombia với 8306 cuộc tấn công
6. Philippines với 6908 cuộc tấn công
7. Peru với 6096 cuộc tấn công
8. El Salvador với 5320 cuộc tấn công
9. United Kingdom với 5235 cuộc tấn công
10. Turkey với 4292 cuộc tấn công


In [23]:
print('Danh sách 5 khu vực bị tấn công nhiều nhất:')
for i in range(0,5):
    print(f"{i+1}. {df.region.value_counts().index[i]} với {df.region.value_counts()[i]} cuộc tấn công")

Danh sách 5 khu vực bị tấn công nhiều nhất:
1. Middle East & North Africa với 50474 cuộc tấn công
2. South Asia với 44974 cuộc tấn công
3. South America với 18978 cuộc tấn công
4. Sub-Saharan Africa với 17550 cuộc tấn công
5. Western Europe với 16639 cuộc tấn công


# Phân tích dữ liệu